In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import nnfs

nnfs.init()

# Generating a random dataset with 100 samples and 5 input features
np.random.seed(42)
X = np.random.randn(100, 5)
y = np.random.randint(0, 3, 100)  # 3 classes

# One-hot encode the labels for TensorFlow model
encoder = OneHotEncoder(sparse_output=False)
y_one_hot = encoder.fit_transform(y.reshape(-1, 1))

# Splitting dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:

# ---------------- Custom Neural Network ----------------

class Layers_Dense:
    def __init__(self, NoOfInputs, NoOfNeurons):
        self.weights = np.random.randn(NoOfInputs, NoOfNeurons) * 0.01  # Small random values
        self.bias = np.zeros((1, NoOfNeurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.bias

    def backward(self, dvalues, learning_rate=0.01):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)

        # Gradient descent update
        self.weights -= learning_rate * self.dweights
        self.bias -= learning_rate * self.dbiases

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs /= samples

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss 

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)  
        correct_confidences = y_pred[range(samples), y_true]
        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs /= samples

In [8]:
# Initializing the custom neural network
dense1 = Layers_Dense(5, 10)  # First layer with 10 neurons
activation_1 = Activation_ReLU()
dense2 = Layers_Dense(10, 3)  # Output layer with 3 neurons (3 classes)
activation_2 = Activation_Softmax()
loss_function = Loss_CategoricalCrossentropy()

In [10]:

# Training for 5 epochs
epochs = 50
learning_rate = 0.01

for epoch in range(epochs):
    # Forward pass
    dense1.forward(X_train)
    activation_1.forward(dense1.output)
    dense2.forward(activation_1.output)
    activation_2.forward(dense2.output)

    # Compute loss
    loss = loss_function.calculate(activation_2.output, y_train)

    # Accuracy calculation
    predictions = np.argmax(activation_2.output, axis=1)
    accuracy_custom_nn = np.mean(predictions == y_train)

    # Backward pass
    loss_function.backward(activation_2.output, y_train)
    activation_2.backward(loss_function.dinputs, y_train)
    dense2.backward(activation_2.dinputs, learning_rate)
    activation_1.backward(dense2.dinputs)
    dense1.backward(activation_1.dinputs, learning_rate)

    print(f"Epoch {epoch+1}/{epochs} - Custom NN Loss: {loss:.4f}, Accuracy: {accuracy_custom_nn:.2f}")


Epoch 1/50 - Custom NN Loss: 1.0983, Accuracy: 0.40
Epoch 2/50 - Custom NN Loss: 1.0983, Accuracy: 0.40
Epoch 3/50 - Custom NN Loss: 1.0982, Accuracy: 0.40
Epoch 4/50 - Custom NN Loss: 1.0981, Accuracy: 0.40
Epoch 5/50 - Custom NN Loss: 1.0981, Accuracy: 0.40
Epoch 6/50 - Custom NN Loss: 1.0980, Accuracy: 0.40
Epoch 7/50 - Custom NN Loss: 1.0980, Accuracy: 0.40
Epoch 8/50 - Custom NN Loss: 1.0979, Accuracy: 0.40
Epoch 9/50 - Custom NN Loss: 1.0979, Accuracy: 0.40
Epoch 10/50 - Custom NN Loss: 1.0978, Accuracy: 0.40
Epoch 11/50 - Custom NN Loss: 1.0978, Accuracy: 0.40
Epoch 12/50 - Custom NN Loss: 1.0978, Accuracy: 0.40
Epoch 13/50 - Custom NN Loss: 1.0977, Accuracy: 0.40
Epoch 14/50 - Custom NN Loss: 1.0977, Accuracy: 0.40
Epoch 15/50 - Custom NN Loss: 1.0976, Accuracy: 0.40
Epoch 16/50 - Custom NN Loss: 1.0976, Accuracy: 0.40
Epoch 17/50 - Custom NN Loss: 1.0975, Accuracy: 0.40
Epoch 18/50 - Custom NN Loss: 1.0975, Accuracy: 0.40
Epoch 19/50 - Custom NN Loss: 1.0974, Accuracy: 0.40
Ep

In [11]:
# ---------------- TensorFlow Model ----------------

# Building a simple dense neural network using TensorFlow
model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(5,)),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:

# Training the TensorFlow model for 5 epochs
model.fit(X_train, y_one_hot[:len(X_train)], epochs=5, verbose=1)

# Evaluate the TensorFlow model
loss_tf, accuracy_tf = model.evaluate(X_test, encoder.transform(y_test.reshape(-1, 1)), verbose=0)

print(f"TensorFlow Neural Network Loss: {loss_tf:.4f}")
print(f"TensorFlow Neural Network Accuracy: {accuracy_tf:.2f}")

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.3000 - loss: 1.4480 
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3023 - loss: 1.4520
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3164 - loss: 1.3530
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3008 - loss: 1.3924 
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3125 - loss: 1.3467
TensorFlow Neural Network Loss: 1.3826
TensorFlow Neural Network Accuracy: 0.20
